### Vertical Index calcultation from preprocessed data tables
In this code we will load the data and calculate the vertical index as
VI=V-H/V+H

Names of files used in session are the ones used to load the correspoding preprocessed data

Lets load the data

In [1]:
import pandas as pd 
import numpy as np
from pathlib import Path
import os
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import signal
import sys
sys.path.append('../')
import commons as cm
from data_curation import Normalization
from pandasql import sqldf

nm=Normalization()

In [2]:
def calculate_contrast(x,y):
    """Function to calculate michealson contrast
    Args:
        x (_np.array_): _description_
        y (_np.arry_): _description_
    """
    contrast=(x-y)/(x+y)
    print(contrast)
    return contrast

def compute_vertical_index(vector_index):
    """Calculate vertical index from a vector of 0 and 1
    Args:
        vector_index (_type_): 0 and ones int numpy array vector
    """
    vertical=vector_index[vector_index==1].size
    horizontal=vector_index[vector_index==0].size
    vi=calculate_contrast(vertical,horizontal)
    print(f'vertical: {vertical}, horizontal: {horizontal},verticality: {vector_index}')
    return vi

In [3]:
session_location = Path(r"P:\Proyectos\2020\PY200016 - XSCAPE\Trabajo\00 XPERIMENTS\01 EXPERIMENTS_Santiago\01 ONGOING\Exp_Stgo_0001\SESSIONS")
data_path=Path(r"data")
sessions=[]
with open('session.txt','r') as f:
    for session in f:
            sessions.append(session.replace('\n',''))
f.close()
session_loc_list=[session_location.joinpath(session,'01 Processing_data','pupil_labs') for session in sessions ]

for subject, recording_location in zip(sessions,session_loc_list):
    #Load data in folders
    recording_folder=[record for record in os.listdir(recording_location)  if '00' in record]
    if len(recording_folder)>1:
        ValueError('Ambiguty in folder of experiment')
    recording_location=recording_location.joinpath(recording_folder[0],'exports')
    recording_location_raw=recording_location.joinpath(recording_folder[0],'exports')
    export_folder=[record for record in os.listdir(recording_location)  if '00' in record]
    if len(export_folder)>1:
        ValueError('Ambiguty in folder of exports')
    recording_location=recording_location.joinpath(export_folder[0])

    print(f'Analiysing session: {subject}')
    annotations_csv = os.path.join(recording_location,'annotations.csv')
    annotations_pd = pd.read_csv(annotations_csv) 
    exported_gaze_csv = data_path.joinpath(subject.replace('_PC_EM',"")+'_saccades.csv') 
    gaze_pd_frame = pd.read_csv(exported_gaze_csv)


Analiysing session: ES0001_S003_PC_EM


### Cut all data by annotations of interest

In [4]:
from IPython.display import display

event_initial=annotations_pd['label'].values[0]
event_final=annotations_pd['label'].values[-1]

initial_anotation,_,_=cm.extract_annotations_timestamps(event_initial,'label',annotations_pd)
end_anotation=annotations_pd.iloc[-1]
display(initial_anotation)
display(end_anotation)

gaze_pd_frame=cm.filter_rows_by_temporal_values(
         dataframe=gaze_pd_frame,
         time_column='start_timestamp',
         ini_value=initial_anotation['timestamp'].values[0],
         end_value=end_anotation['timestamp']
         )


,index,timestamp,label,duration
0,6037,7977.395531,blank_0,NaN


index                  38265
timestamp        8517.385911
label        EndOfExperiment
duration                 NaN
Name: 162, dtype: object

## Extract the data of interest
Removing out of surface events

In [5]:
gaze_pd_frame=gaze_pd_frame.query('on_surf==True')
gaze_pd_frame['on_surf'].unique()

array([ True])

Order events

In [6]:
filter_events=annotations_pd['label'].str.contains('Asset') | annotations_pd['label'].str.contains('Control') | annotations_pd['label'].str.contains('Surprise')
event=annotations_pd.loc[filter_events,['label']].values.flatten().tolist()
event.sort()
event_strip=[image.split('.')[0] for image in event] #remove .tiiff
# event.sort(key=lambda x: int(x[x.index('.')-2:x.index('.')]))


## Calculate Vertical Index

In [7]:
data_dict=dict([(key,[None]) for key in event_strip])# dict with empty keys 
vertical_index_df=pd.DataFrame()#pd.DataFrame(data_dict,index=np.arange(0,800))
data_list=[]
for im,im_strip in zip(event,event_strip):
    initial_anotation,end_anotation,index_annotation=cm.extract_annotations_timestamps(im,'label',annotations_pd)
    segmented_df=cm.filter_rows_by_temporal_values(
        dataframe=gaze_pd_frame,
        time_column='start_timestamp',
        ini_value=initial_anotation['timestamp'].values[0],
        end_value=end_anotation['timestamp'].values[0]
        )   
    display(segmented_df)
    verticality=segmented_df['verticality'].values
    vi=compute_vertical_index(verticality)
    data_dict[im_strip]=[vi]
vertical_index_df=pd.DataFrame(data_dict)
vertical_index_df.index=[subject]

,Unnamed: 0,world_timestamp,world_index,fixation_id,start_timestamp,duration,dispersion,norm_pos_x,norm_pos_y,x_scaled,y_scaled,on_surf,amplitude,speed,angle,orientation,verticality
591,591,8201.409109,19406,877,8201.410304,333.9205,0.641670,0.513854,0.527504,0.513854,0.527504,True,0.001914,0.003799,163.013868,163.013868,0.0
592,592,8201.809125,19430,878,8201.812278,122.0115,1.448092,0.391299,0.570552,0.391299,0.570552,True,0.129896,0.324726,160.646385,160.646385,0.0
593,593,8201.945130,19438,879,8201.946201,314.0820,1.471231,0.398623,0.665257,0.398623,0.665257,True,0.094989,0.698421,85.577659,85.577659,1.0
594,594,8202.261127,19457,880,8202.264202,158.0945,0.597044,0.402915,0.677526,0.402915,0.677526,True,0.012998,0.041133,70.719873,70.719873,1.0
595,595,8202.461148,19469,881,8202.460473,203.8115,1.348277,0.293670,0.520308,0.293670,0.520308,True,0.191447,0.957136,-124.793836,235.206164,1.0
596,596,8202.681321,19482,882,8202.680286,224.1185,1.424918,0.381608,0.454389,0.381608,0.454389,True,0.109902,0.499162,-36.855573,323.144428,0.0
597,597,8202.965224,19499,883,8202.964314,183.8945,1.466317,0.280228,0.444726,0.280228,0.444726,True,0.101840,0.358715,-174.555494,185.444506,0.0
598,598,8203.233247,19515,884,8203.238311,178.0665,1.444477,0.537665,0.321905,0.537665,0.321905,True,0.285235,1.064220,-25.505290,334.494710,0.0
599,599,8203.453369,19528,885,8203.448316,131.8870,1.490636,0.634382,0.406385,0.634382,0.406385,True,0.128417,0.583392,41.136584,41.136584,0.0
600,600,8203.701232,19543,886,8203.698279,123.9825,1.394265,0.676971,0.560492,0.676971,0.560492,True,0.159883,0.645048,74.551399,74.551399,1.0


0.2
vertical: 12, horizontal: 8,verticality: [0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1.]


,Unnamed: 0,world_timestamp,world_index,fixation_id,start_timestamp,duration,dispersion,norm_pos_x,norm_pos_y,x_scaled,y_scaled,on_surf,amplitude,speed,angle,orientation,verticality
565,565,8191.773201,18831,851,8191.768241,160.0320,0.957757,0.518211,0.607339,0.518211,0.607339,True,0.047795,0.094868,79.972881,79.972881,1.0
566,566,8191.941375,18841,852,8191.948227,150.0905,1.486536,0.439089,0.648987,0.439089,0.648987,True,0.089413,0.531671,152.238665,152.238665,0.0
567,567,8192.193182,18856,853,8192.196456,255.8660,1.432425,0.362092,0.668299,0.362092,0.668299,True,0.079383,0.315252,165.919626,165.919626,0.0
568,568,8192.461164,18872,854,8192.456313,499.9245,0.610009,0.346225,0.668938,0.346225,0.668938,True,0.015880,0.059257,177.692212,177.692212,0.0
569,569,8192.961126,18902,855,8192.960221,498.0685,0.506290,0.350565,0.664613,0.350565,0.664613,True,0.006128,0.012256,-44.897141,315.102859,0.0
570,570,8193.465247,18932,856,8193.462303,142.1125,0.456857,0.348240,0.655978,0.348240,0.655978,True,0.008943,0.017740,-105.072842,254.927158,1.0
571,571,8193.617265,18941,857,8193.620340,183.9570,1.425594,0.435022,0.675679,0.435022,0.675679,True,0.088991,0.585397,12.790538,12.790538,0.0
572,572,8193.885258,18957,858,8193.888299,110.0145,1.443318,0.512299,0.622785,0.512299,0.622785,True,0.093645,0.349432,-34.390707,325.609293,0.0
573,573,8194.001287,18964,859,8194.002307,498.0555,1.223190,0.520439,0.581503,0.520439,0.581503,True,0.042077,0.362641,-78.845788,281.154212,1.0
574,574,8194.505144,18994,860,8194.504413,499.9085,0.481184,0.506757,0.587925,0.506757,0.587925,True,0.015113,0.029995,154.857135,154.857135,0.0


-0.1111111111111111
vertical: 8, horizontal: 10,verticality: [1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1.]


,Unnamed: 0,world_timestamp,world_index,fixation_id,start_timestamp,duration,dispersion,norm_pos_x,norm_pos_y,x_scaled,y_scaled,on_surf,amplitude,speed,angle,orientation,verticality
1289,1289,8451.728898,34346,1575,8451.736047,104.0085,1.420927,0.503293,0.609891,0.503293,0.609891,True,0.087482,0.154036,83.432441,83.432441,1.0
1290,1290,8451.844883,34353,1576,8451.844036,347.9835,1.268993,0.510853,0.638075,0.510853,0.638075,True,0.029181,0.251592,74.984760,74.984760,1.0
1291,1291,8452.217101,34375,1577,8452.208024,162.0020,1.458531,0.497175,0.742462,0.497175,0.742462,True,0.105279,0.282843,97.465072,97.465072,1.0
1292,1292,8452.364939,34384,1578,8452.374040,150.1020,1.441269,0.486248,0.760008,0.486248,0.760008,True,0.020670,0.139813,121.912964,121.912964,1.0
1293,1293,8452.532908,34394,1579,8452.528082,175.9325,1.346350,0.456504,0.766725,0.456504,0.766725,True,0.030493,0.181538,167.274106,167.274106,0.0
1294,1294,8452.801028,34410,1580,8452.800009,212.0300,1.444043,0.335302,0.762918,0.335302,0.762918,True,0.121262,0.452269,-178.200999,181.799001,0.0
1295,1295,8453.252997,34437,1581,8453.252015,113.9305,1.496976,0.484214,0.497021,0.484214,0.497021,True,0.304756,0.674286,-60.749597,299.250403,1.0
1296,1296,8453.456868,34449,1582,8453.463981,152.0525,1.363182,0.350942,0.538767,0.350942,0.538767,True,0.139658,0.685029,162.607389,162.607389,0.0
1297,1297,8453.621032,34459,1583,8453.624020,496.0610,1.187907,0.315534,0.539281,0.315534,0.539281,True,0.035411,0.215707,179.168457,179.168457,0.0
1298,1298,8454.124919,34489,1584,8454.124061,105.9605,0.507777,0.319117,0.554467,0.319117,0.554467,True,0.015603,0.030966,76.724258,76.724258,1.0


...

0.47368421052631576
vertical: 14, horizontal: 5,verticality: [1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1.]


,Unnamed: 0,world_timestamp,world_index,fixation_id,start_timestamp,duration,dispersion,norm_pos_x,norm_pos_y,x_scaled,y_scaled,on_surf,amplitude,speed,angle,orientation,verticality
539,539,8181.653293,18227,825,8181.656294,344.0425,1.499982,0.602900,0.580794,0.602900,0.580794,True,0.139651,0.363743,48.391137,48.391137,1.0
540,540,8182.021269,18249,826,8182.020331,376.0635,1.476648,0.549890,0.526939,0.549890,0.526939,True,0.075567,0.205359,-134.547272,225.452728,1.0
541,541,8182.405165,18272,827,8182.400434,499.9600,1.223369,0.556970,0.559824,0.556970,0.559824,True,0.033638,0.087623,77.849630,77.849630,1.0
542,542,8182.909168,18302,828,8182.904391,365.8785,0.950092,0.540728,0.565327,0.540728,0.565327,True,0.017149,0.034027,161.280855,161.280855,0.0
543,543,8183.293159,18325,829,8183.288404,316.0990,1.389234,0.606013,0.623577,0.606013,0.623577,True,0.087494,0.227854,41.740138,41.740138,0.0
544,544,8183.613141,18344,830,8183.610487,229.8395,1.471482,0.604456,0.634449,0.604456,0.634449,True,0.010983,0.034324,98.151341,98.151341,1.0
545,545,8183.849277,18358,831,8183.844343,227.9580,0.280889,0.615753,0.676730,0.615753,0.676730,True,0.043764,0.185334,75.040313,75.040313,1.0
546,546,8184.149156,18376,832,8184.146241,106.7325,0.237805,0.617704,0.687819,0.617704,0.687819,True,0.011260,0.037548,80.024840,80.024840,1.0
547,547,8184.369391,18389,833,8184.368215,164.0995,1.450684,0.557854,0.599598,0.557854,0.599598,True,0.106607,0.484059,-124.153218,235.846782,1.0
548,548,8184.553291,18400,834,8184.558271,103.9710,0.305856,0.553146,0.607128,0.553146,0.607128,True,0.008881,0.048292,122.015166,122.015166,1.0


0.0
vertical: 9, horizontal: 9,verticality: [1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0.]


,Unnamed: 0,world_timestamp,world_index,fixation_id,start_timestamp,duration,dispersion,norm_pos_x,norm_pos_y,x_scaled,y_scaled,on_surf,amplitude,speed,angle,orientation,verticality
1116,1116,8391.697053,30763,1402,8391.690095,214.0105,1.486855,0.496039,0.670688,0.496039,0.670688,True,0.110176,0.252633,94.542801,94.542801,1.0
1117,1117,8391.913052,30776,1403,8391.920092,496.0275,1.320640,0.448394,0.660672,0.448394,0.660672,True,0.048686,0.225399,-168.127624,191.872376,0.0
1118,1118,8392.417121,30806,1404,8392.420133,103.9715,1.075567,0.447742,0.651132,0.447742,0.651132,True,0.009563,0.018971,-93.913790,266.086210,1.0
1119,1119,8392.569059,30815,1405,8392.566009,130.0980,1.434450,0.507947,0.508211,0.507947,0.508211,True,0.155084,1.020707,-67.156790,292.843210,1.0
1120,1120,8392.701047,30823,1406,8392.700101,325.9785,1.468922,0.515449,0.462511,0.515449,0.462511,True,0.046311,0.350875,-80.677658,279.322342,1.0
1121,1121,8393.037041,30843,1407,8393.030099,499.9775,0.641274,0.532079,0.438605,0.532079,0.438605,True,0.029121,0.086672,-55.176227,304.823773,1.0
1122,1122,8393.640916,30879,1408,8393.644121,167.9725,1.302742,0.589890,0.562421,0.589890,0.562421,True,0.136647,0.226284,64.971393,64.971393,1.0
1123,1123,8393.824927,30890,1409,8393.816084,137.9390,1.384364,0.585913,0.627065,0.585913,0.627065,True,0.064766,0.351968,93.520764,93.520764,1.0
1124,1124,8393.956947,30898,1410,8393.958001,126.1285,1.431357,0.575483,0.667406,0.575483,0.667406,True,0.041668,0.315620,104.495471,104.495471,1.0
1125,1125,8394.092919,30906,1411,8394.088175,147.9025,1.216681,0.563732,0.719212,0.563732,0.719212,True,0.053121,0.390679,102.780173,102.780173,1.0


0.42857142857142855
vertical: 15, horizontal: 6,verticality: [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0.]


In [164]:
vertical_df

,Asset0032.tif,Asset0044.tif,Asset0049.tif,Asset0005.tif,Asset0037.tif,Asset0006.tif,Asset0017.tif,Asset0022.tif,Control01.tif,Surprise02.tif,...,Control04.tif,Asset0039.tif,Asset0027.tif,Asset0003.tif,Asset0011.tif,Asset0021.tif,Asset0029.tif,Asset0012.tif,Asset0007.tif,Asset0046.tif
ES0001_S003_PC_EM,0.428571,0.0,0.294118,0.111111,0.125,-0.263158,0.333333,-0.222222,-0.5,0.473684,...,0.333333,0.142857,0.125,0.12,-0.058824,-0.058824,-0.1,0.238095,0.111111,0.052632


In [16]:
vertical_index_df['Control04']

ES0001_S003_PC_EM    0.333333
Name: Control04, dtype: float64